In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
--2018-12-10 12:02:12--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.223, 91.189.89.222
Connecting to launchpad.net (launchpad.net)|91.189.89.223|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb [following]
--2018-12-10 12:02:13--  https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpadlibrarian.net (launchpadlibrarian.net)... 91.189.89.229, 91.189.89.228
Connecting to launchpadlibrarian.net (launchpadlibrarian.net)|91.189.89.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1232624 (1.2M) [application/x-debian-package]
Saving to: ‘google-drive-ocamlfuse_

In [2]:
!fusermount -u drive
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty
!ls drive

fusermount: failed to unmount /content/drive: No such file or directory
 armin			   DL			      louvre_tickets.pdf
'budget planing'	   DL_Slide-uri_1.pdf	      model.json
 codiax			   DL_Slide-uri_2.pdf	      model_weights_v3.h5
'Colab Notebooks'	   Documents		     'Orar anul 4.ods'
 contacts11.09.2017.vcf    eiffel_tower_tickets.pdf   SafeInCloud.db
'Copy of SafeStreet.pdf'   Facultate		      SafeStreet.pdf
 crypto			   Garbage		     'Untitled document.odt'
 D			   hacktm2018sibiu
 dataset_imp		   licenta


In [0]:
from google.colab import files
src = list(files.upload().values())[0]
open('drive/licenta/utils.py','wb').write(src)
import drive.licenta.utils

IndexError: ignored

In [3]:
import pandas as pd # data analysis toolkit - create, read, update, delete datasets
import numpy as np #matrix math
from sklearn.model_selection import train_test_split #to split out training and testing data 
from keras.models import Sequential, Model
from keras.optimizers import Adam
#to save our model periodically as checkpoints for loading later
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Input
#helper class to define input shape and generate training images given image paths & steering angles
from keras.backend import tensorflow_backend as backend
from drive.licenta.utils import INPUT_SHAPE, batch_generator
import matplotlib.pyplot as plt

#for debugging, allows for reproducible (deterministic) results 
np.random.seed(0)

model = Sequential()

batch_size = 256
epochs = 50
steps_per_epoch = 8

root_path = 'drive/licenta/data/'

checkpoint_path = 'drive/licenta/model/model4.hdf5'

csv_train_path = root_path + 'train/CSV/train_driving_log_map_2_attempts3.csv'
csv_validation_path = root_path + 'validation/CSV/validation_driving_log_map2_attempts2.csv'
csv_test_path = root_path + 'test/CSV/test_driving_log_map2_attempts1.csv'

optimizer=Adam(lr=0.0001)

def load_csv(path):
  #reads CSV file into a single dataframe variable
  data_df = pd.read_csv(path, names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])
  #we'll store the camera images as our input data
  X = data_df[['center', 'left', 'right']].values

  #and our steering commands as our output data
  Y = data_df['steering'].values
  
  return X, Y

def load_data():
  """
  Load training data and split it into training and validation set
  """
  X_train, Y_train = load_csv(csv_train_path)
  X_valid, Y_valid = load_csv(csv_validation_path)
  X_test, Y_test = load_csv(csv_test_path)

  #now we can split the data into a training & validation
  #thanks scikit learn
  #X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=X[0].size, random_state=0)

  return X_train, Y_train, X_valid, Y_valid, X_test, Y_test


def build_model():
  """
  NVIDIA model used
  Image normalization to avoid saturation and make gradients work better.
  Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
  Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
  Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
  Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
  Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
  Drop out (0.5)
  Fully connected: neurons: 100, activation: ELU
  Fully connected: neurons: 50, activation: ELU
  Fully connected: neurons: 10, activation: ELU
  Fully connected: neurons: 1 (output)

  # the convolution layers are meant to handle feature engineering
  the fully connected layer for predicting the steering angle.
  dropout avoids overfitting
  ELU(Exponential linear unit) function takes care of the Vanishing gradient problem. 
  """

#Use the generated model 
  inputs = Input(shape=INPUT_SHAPE)
  
  model = Sequential()
  model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
  model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
  model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
  model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
  model.add(Conv2D(64, 3, 3, activation='elu'))
  model.add(Conv2D(64, 3, 3, activation='elu'))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(100, activation='elu'))
  model.add(Dense(50, activation='elu'))
  model.add(Dense(10, activation='elu'))
  model.add(Dense(1))
  
  model.summary()

  return model


def plot_history(history):
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()
  # "Loss"
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()
  
    
def evaluate_model(history, X_test, Y_test):
  plot_history(history)
   
  test_batches = batch_generator(X_test,
                                 Y_test, 
                                 batch_size, 
                                 is_training=False)
  
  # Evaluate model
  score = history.model.evaluate_generator(test_batches, steps=1)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])

  # Clear session
  clear_session();
  
    
def train_model(model, X_train, Y_train, X_valid, Y_valid):
  """
  Train the model
  """

  checkpoint = ModelCheckpoint(checkpoint_path, 
                               monitor='val_acc', 
                               save_best_only=True, 
                               mode='auto',
                               verbose=1)
  
  callbacks=[checkpoint]

  model.compile(loss='mean_squared_error', 
                optimizer=optimizer,
                metrics=['acc'])
  
  train_batches = batch_generator(X_train, 
                                  Y_train, 
                                  batch_size,
                                  is_training=True)
  
  validation_batches = batch_generator(X_valid, 
                                       Y_valid, 
                                       batch_size, 
                                       is_training=False)
  
  history = model.fit_generator(train_batches,
                                validation_data=validation_batches,
                                steps_per_epoch=steps_per_epoch,
                                validation_steps=steps_per_epoch,
                                epochs=epochs,
                                callbacks=callbacks,
                                verbose=1)
  return history


def clear_session():
  backend.clear_session()
  
  
def main():
  """
  Load train/validation data set and train the model
  """

  #load data
  X_train, Y_train, X_valid, Y_valid, X_test, Y_test = load_data()
  train_valid_data = X_train, Y_train, X_valid, Y_valid
  test_data = X_test, Y_test
  #build model
  model = build_model()
  #train model on data, it saves as model.h5 
  history = train_model(model, *train_valid_data)
  model = history.model
  evaluate_model(history, *test_data)
  
  
if __name__ == '__main__':
  main()



Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 18, 64)         0         
__________

TypeError: ignored

In [0]:
import matplotlib.pyplot as plt
model = Sequential()

def load_trained_model(weights_path):
  model = build_model()
  model.load_weights(weights_path)
  
load_trained_model(checkpoint_path)

X_train, Y_train, X_valid, Y_valid, X_test, Y_test = load_data()
model.compile(loss='mean_squared_error', 
                optimizer=optimizer,
                metrics=['acc'])
model.evaluate(X_test, Y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_30 (Lambda)           (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_146 (Conv2D)          (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_147 (Conv2D)          (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_148 (Conv2D)          (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_149 (Conv2D)          (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_150 (Conv2D)          (None, 1, 18, 64)         36928     
_________________________________________________________________
dropout_30 (Dropout)         (None, 1, 18, 64)         0         
__________

ValueError: ignored